In [1]:
%load_ext autoreload

In [1]:
import numpy as np
import os
import mcubes
import meshcat
import pydrake
from pydrake.geometry import SceneGraph
from pydrake.systems.framework import DiagramBuilder
from pydrake.common import FindResourceOrThrow
from pydrake.multibody.plant import MultibodyPlant, AddMultibodyPlantSceneGraph
from pydrake.multibody.parsing import Parser, LoadModelDirectives, ProcessModelDirectives
from pydrake.multibody.tree import RevoluteJoint
from pydrake.all import ConnectMeshcatVisualizer, InverseKinematics, RigidTransform, RotationMatrix
from pydrake.all import BsplineTrajectoryThroughUnionOfHPolyhedra, IrisInConfigurationSpace, IrisOptions, Rgba
import time
from meshcat import Visualizer
from functools import partial

import ipywidgets as widgets
from IPython.display import display

from pydrake.all import MeshcatVisualizerCpp, MeshcatVisualizerParams, Role
import rrt, prm, utils, rrtiris
from pydrake.all import (MathematicalProgram, Variable, HPolyhedron, le, SnoptSolver, Solve) 


In [2]:
# Setup meshcat
from meshcat.servers.zmqserver import start_zmq_server_as_subprocess
proc, zmq_url, web_url = start_zmq_server_as_subprocess(server_args=[])
proc2, zmq_url2, web_url2 = start_zmq_server_as_subprocess(server_args=[])

In [3]:
def meshcat_line(x_start, x_end, width):
    x_end_shift = x_end.copy()
    x_end_shift[0:2] += width
    x_start_shift = x_start.copy()
    x_start_shift[1:2] += width
    
    points = np.array([[x_start, x_end, x_end_shift, x_start_shift]]).reshape(-1,3)
    triangles = np.array([[0,1,2],[0,2,3]]).reshape(-1,3)
    mc_geom = meshcat.geometry.TriangularMeshGeometry(points, triangles)
    return mc_geom

In [4]:
#settings
q0 = [0.0, 0.0, 0.0]
q_low = [-1.7,-1.7, -2.0]
q_high = [1.7, 1.7, 2.0]

#marching cubes
q_low_mc = q_low.copy()
q_high_mc =  q_high.copy()
N = 50

In [5]:
vis = Visualizer(zmq_url=zmq_url)
vis.delete()
vis2 = Visualizer(zmq_url=zmq_url2)
vis2.delete()

builder = DiagramBuilder()
plant, scene_graph = AddMultibodyPlantSceneGraph(builder, time_step=0.001)
parser = Parser(plant)
tworob_asset = FindResourceOrThrow("drake/sandbox/assets/doublerob.urdf")
#two_dof_asset = FindResourceOrThrow("drake/sandbox/assets/planar2dof.urdf")
box_asset = FindResourceOrThrow("drake/sandbox/assets/box.urdf")

models =[]
models.append(parser.AddModelFromFile(tworob_asset))
#models.append(parser.AddModelFromFile(one_dof_asset))
models.append(parser.AddModelFromFile(box_asset))

idx = 0
for model in models:
    for joint_index in plant.GetJointIndices(model):
        joint = plant.get_mutable_joint(joint_index)
        if isinstance(joint, RevoluteJoint):
            joint.set_default_angle(q0[idx])
            idx += 1
            
locs = [[0.,0.,0.],[0.,0.,0.]]
idx = 0
for model in models:
    plant.WeldFrames(plant.world_frame(), plant.GetFrameByName("base", model), RigidTransform(locs[idx]))
    idx+=1

plant.Finalize()

visualizer = ConnectMeshcatVisualizer(builder, scene_graph, zmq_url=zmq_url, delete_prefix_on_load=False, )

diagram = builder.Build()
visualizer.load()
context = diagram.CreateDefaultContext()
plant_context = plant.GetMyContextFromRoot(context)
diagram.Publish(context)

joints = []
idx = 0
for model in models:
    jointindx = plant.GetJointIndices(model)
    for j in jointindx:
        joint = plant.get_mutable_joint(j)
        if isinstance(joint, RevoluteJoint):
            joints.append(joint)
            joints[-1].set_position_limits(lower_limits= np.array([q_low[idx]]), upper_limits= np.array([np.array([q_high[idx]])]))
            idx +=1
        
    
def set_joint_ang(val, idx):
    joints[idx].set_angle(plant_context, val)
    
def set_joint_angles(vals):
    joints[0].set_angle(plant_context, vals[0])
    joints[1].set_angle(plant_context, vals[1])
    joints[2].set_angle(plant_context, vals[2])
    
ik = InverseKinematics(plant, plant_context)
collision_constraint = ik.AddMinimumDistanceConstraint(0.001, 0.01)

def eval_cons(q0, q1, q2, c, tol):
        return 1-1*float(c.evaluator().CheckSatisfied([q0, q1, q2], tol))
    
col_func_handle = partial(eval_cons, c=collision_constraint, tol=0.01)

def showres(q):
    set_joint_ang(q[0],0)
    set_joint_ang(q[1],1)
    set_joint_ang(q[2],2)
    col = col_func_handle(*q)
    if col:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0xFFB900))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(q))
    else:
        vis2["q"].set_object(
                meshcat.geometry.Sphere(0.1), meshcat.geometry.MeshLambertMaterial(color=0x3EFF00))
        vis2["q"].set_transform(
                meshcat.transformations.translation_matrix(q))
    diagram.Publish(context)
    print("              ", end = "\r")
    print(col , end = "\r")

sliders = []
sliders.append(widgets.FloatSlider(min=q_low[0], max=q_high[0], value=0, description='q0'))
sliders.append(widgets.FloatSlider(min=q_low[1], max=q_high[1], value=0, description='q1'))
sliders.append(widgets.FloatSlider(min=q_low[2], max=q_high[2], value=0, description='q2'))

q = q0.copy()
def handle_slider_change(change, idx):
    q[idx] = change['new']
    #print(q, end="\r")
    showres(q)
    
idx = 0
for slider in sliders:
    slider.observe(partial(handle_slider_change, idx = idx), names='value')
    idx+=1


You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7003/static/
Connecting to meshcat-server at zmq_url=tcp://127.0.0.1:6002...
You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/
Connected to meshcat-server.


In [6]:
#marching cubes
vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), N, N, N, col_func_handle, 0.5)
vis2["collision_constraint"].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            meshcat.geometry.MeshLambertMaterial(color=0xff0000, wireframe=True))
q = q0.copy()
showres(q)

In [7]:
for slider in sliders:
    display(slider)

display(vis.jupyter_cell())
display(vis2.jupyter_cell())

FloatSlider(value=0.0, description='q0', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q1', max=1.7, min=-1.7)

FloatSlider(value=0.0, description='q2', max=2.0, min=-2.0)

In [8]:
#SPP + IRIS CONFIG
start = np.array([0.2, -1.6, 1.5])
target = np.array([0.8,-0.9,1.5])
#
#plot start and target
mat = meshcat.geometry.MeshLambertMaterial(color=0xFFDD36)
mat.reflectivity = 1.0
vis2['start'].set_object(
                meshcat.geometry.Sphere(0.06), mat)
vis2['start'].set_transform(
                meshcat.transformations.translation_matrix(start.reshape(-1,)))

mat = meshcat.geometry.MeshLambertMaterial(color=0x06D300)
mat.reflectivity = 1.0
vis2['target'].set_object(
                meshcat.geometry.Sphere(0.06), mat)
vis2['target'].set_transform(
                meshcat.transformations.translation_matrix(target.reshape(-1,)))

In [15]:
def Iris_fn(q_seed, iris_options, plant, plant_context, verbose):
    start_time = time.time()
    region = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    if verbose:
        print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
  
    return region, ellipse

IRISOPTS = IrisOptions()
IRISOPTS.require_sample_point_is_contained = True
IRISOPTS.iteration_limit = 60
IRISOPTS.enable_ibex = False

iris_hand = partial(Iris_fn, 
                    iris_options = IRISOPTS, 
                    plant = plant, 
                    plant_context = plant_context, 
                    verbose = False)

def plot_callback(region, seed_point, pos_samp, region_id, vis):
    scale = np.clip(150 + region_id *105/5.0, a_min = 0, a_max = 255)
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((int(0.2*scale), 155, int(scale))) , wireframe=True)
    mat_seed = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((0, 0, 0)), wireframe=True)
    mat_samp = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((155, 0, 155)), wireframe=True)
    mat.opacity = 0.2
    
    #print('seed_point', seed_point)
    #print('pos_samp', pos_samp)
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis['regions'],
                       name = str(region_id),
                       mat = mat)
    utils.plot_point(seed_point,
                     radius = 0.05, 
                     mat = mat_seed, 
                     vis = vis['seed'], 
                     marker_id = str(region_id))
    
    utils.plot_point(pos_samp,
                     radius = 0.05, 
                     mat = mat_samp, 
                     vis = vis['samples'], 
                     marker_id = str(region_id))
    
plot_callback_handle = partial(plot_callback, vis = vis2['rrtiris']) 

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrti_col_fn_handle = partial(collision, col_func_handle = col_func_handle)

def get_closest_point_in_set(polys, sample, dim = 3):
    num_regions = len(polys) 
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(dim*num_regions, 'x')  
    cost = x - np.tile(sample, (1, num_regions)).squeeze()
    cost = cost@cost.T
    prog.AddCost(cost)
    for idx in range(num_regions):
        A = polys[idx].A()
        b = polys[idx].b()
        prog.AddConstraint(le(A@x[idx*dim:(idx + 1)*dim], b))
    solver = SnoptSolver()
    result = solver.Solve(prog)
    x_sol = result.GetSolution(x).reshape(num_regions,3)
    dists = np.linalg.norm(x_sol-sample, axis = 1)
    closest = np.argmin(dists)
    min_dist = dists[closest]
    min_point = x_sol[closest, :]
    
    return min_point, min_dist, x_sol, result

In [24]:
RRTI = rrtiris.RRTIRIS(start = start, 
                       goal = target , 
                       limits = [np.array(q_low), np.array(q_high)], 
                       iris_handle = iris_hand,
                       col_func_handle = rrti_col_fn_handle, 
                       init_goal_sample_rate = 0.05,
                       goal_sample_rate_scaler = 0.6,
                       verbose = True,
                       plotcallback = plot_callback_handle,
                       sample_collision_free = True
                       )

In [25]:
RRTI.run(40)

0
0.6133742174409513
[RRT IRIS] it: 0 distance to target:  0.613 goalsample prob:  0.432
1
[RRT IRIS] Region check failed
2
0.6133742174409513
3
0.6133742174409513
4
[RRT IRIS] Region check failed
5
[RRT IRIS] Region check failed
6
[RRT IRIS] Region check failed
7
0.5946104626764058
[RRT IRIS] it: 7 distance to target:  0.595 goalsample prob:  0.443
8
0.5946104626764057
[RRT IRIS] it: 8 distance to target:  0.595 goalsample prob:  0.443
9
[RRT IRIS] Region check failed
10
0.5946104626764057
11
0.5946104626764057
12
[RRT IRIS] Region check failed
13
0.5946104626764058
14
[RRT IRIS] Region check failed
15
0.5946104626764057
16
0.5946104626764054
[RRT IRIS] it: 16 distance to target:  0.595 goalsample prob:  0.443
17
0.5946104626764057
18
0.5515847685415184
[RRT IRIS] it: 18 distance to target:  0.552 goalsample prob:  0.469
19
0.262962282806295
[RRT IRIS] it: 19 distance to target:  0.263 goalsample prob:  0.642
20
1.1102230246251565e-16
[RRT IRIS] it: 20 distance to target:  0.000 goals

In [12]:
regions = RRTI.node_regions

In [9]:
#using hand-crafted seed points

seed_points = np.array([[0.0, 0, 0], # startpoint
                        [0.8, -0.8, 1.3],  # blue low green up
                        [0.1, -1.2, 0.9],     # green low other up
                        [0.2, -0.6, 1.6],
                        [-0.5, -1.0, 1.9]])    # passing


iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
for i in range(seed_points.shape[0]):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, seed_points[i,:], iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    ellipses.append(ellipse)
    regions.append(hpoly)

Time:   0.17 	Volume:   1.52 	Center: [ 0.19923911 -0.51555715 -0.16204128]
Time:   0.06 	Volume:   0.13 	Center: [ 0.67937999 -0.49625952  1.53735375]
Time:   0.06 	Volume:   1.05 	Center: [-0.06171656 -0.89319338  0.39752367]
Time:   0.09 	Volume:   0.33 	Center: [ 0.05710662 -0.34521315  1.65147226]
Time:   0.05 	Volume:   0.22 	Center: [-0.26468704 -0.33151403  1.54513003]


In [10]:
def do_iris(q_seed, iris_options):
    start_time = time.time()
    hpoly = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = hpoly.MaximumVolumeInscribedEllipsoid()
    print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
    return ellipse, hpoly

q_low_np = np.array(q_low)
q_high_np = np.array(q_high)

iris_options = IrisOptions()
iris_options.require_sample_point_is_contained = True
iris_options.iteration_limit = 50
iris_options.enable_ibex = False

regions = []
ellipses = []
its = 6
seed_points = [start, target]

for point in seed_points:
    ell, reg = do_iris(point, iris_options)
    regions.append(reg)
    ellipses.append(ell)
    
for _ in range(its):
    #rejection sampling to get initial feasible point 
    found = False
    while not found:
        t = np.random.rand(3)
        q_samp = (1-t)*q_low_np + t*q_high_np
        found = (col_func_handle(*q_samp)==0.0)
    print("point found: ", q_samp)
    ell, reg = do_iris(q_samp, iris_options)
    if ell.Volume() < 10.0:
        regions.append(reg)
        ellipses.append(ell)
    

Time:   0.05 	Volume:   0.69 	Center: [-0.18154991 -1.03678815  1.14742869]
Time:   0.07 	Volume:   0.13 	Center: [ 0.64658816 -0.49381788  1.57702836]
point found:  [-0.33279049  0.03732353  0.64922573]
Time:   0.06 	Volume:   0.98 	Center: [ 1.47312677e-01 -3.38797335e-01  1.08180820e-08]
point found:  [ 0.38853657 -1.62600287  0.24983434]
Time:   0.07 	Volume:   0.77 	Center: [-0.1291927  -1.25663458  0.7897659 ]
point found:  [ 0.79480751 -0.00281403 -0.56437643]
Time:   0.17 	Volume:   1.53 	Center: [ 0.20819098 -0.50771476 -0.16374822]
point found:  [ 0.04737768 -0.70436218  1.79877101]
Time:   0.09 	Volume:   0.32 	Center: [ 0.05491061 -0.35492568  1.68583447]
point found:  [ 0.92672521 -0.9014642  -1.8933403 ]
Time:   0.11 	Volume:   1.14 	Center: [ 0.21656636 -0.42985089 -0.10443147]
point found:  [ 0.85916255  0.02108209 -1.592084  ]
Time:   0.20 	Volume:   1.53 	Center: [ 0.20749785 -0.50769877 -0.16216311]


In [11]:
def inpolycheck(q0,q1,q2, A, b):
    q = np.array([q0, q1, q2])
    res = np.min(1.0*(A@q-b<=0))
    #print(res)
    return res

meshes = []
col1 = 0x002BFF
col2 = 0x3EFF00 
idx = 0
step = 1/(1.0*len(regions))

for region in regions:
    A = region.A()
    b = region.b()
    col_hand = partial(inpolycheck, A=A, b=b)
    vertices, triangles = mcubes.marching_cubes_func(tuple(q_low), tuple(q_high), 50, 50, 50, col_hand, 0.5)
    regstr = "regions" +str(idx)
    ellstr = "ellipse" +str(idx)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.3
    vis2['regions'][regstr].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)
    
    C = ellipses[idx].A()
    d = ellipses[idx].center()
    radii, R = np.linalg.eig(C.T@C)
    R[:,0] = R[:,0]*np.linalg.det(R)
    Rot = RotationMatrix(R)
 
    transf = RigidTransform(Rot, d)
    mat = meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True)
    mat.opacity = 0.15
    vis2['ellipses'][ellstr].set_object(
            meshcat.geometry.Ellipsoid(np.divide(1,np.sqrt(radii))),
            meshcat.geometry.MeshLambertMaterial(color= int((1-idx*step)*col1 + idx*step*col2), wireframe=True))

    vis2['ellipses'][ellstr].set_transform(transf.GetAsMatrix4())
    
    idx+=1

In [13]:
# Solve path planning

start_time = time.time()
spp = BsplineTrajectoryThroughUnionOfHPolyhedra(start, target, regions)
spp.set_max_velocity([.8, .8, .8])
spp.set_extra_control_points_per_region(3)

# print(spp.num_regions())
traj = spp.SolveVerbose()
print(time.time() - start_time)
print(traj.start_time())
print(traj.end_time())
#visualize
maxit = 60
pts = []
for it in range(maxit):
    pts.append(traj.value(it*traj.end_time()/maxit))
    mat = meshcat.geometry.MeshLambertMaterial(color=0xFFF812)
    mat.reflectivity = 1.0
    vis2['traj']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.06), mat)
    vis2['traj']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(pts[-1].reshape(-1,)))
    
    set_joint_angles(pts[-1].reshape(-1,))
    tf_l2 = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(3)))
    R_l2 = tf_l2.rotation()
    tl_l2 = R_l2@np.array([0,0,0.9]) + tf_l2.translation()
    
    tf_la = plant.EvalBodyPoseInWorld(plant_context, plant.get_body(pydrake.multibody.tree.BodyIndex(4)))
    R_la = tf_la.rotation()
    tl_la = R_la@np.array([0,0,1.2]) + tf_la.translation()
     
    
    mat = meshcat.geometry.MeshLambertMaterial(color=0x0029F1)
    mat.reflectivity = 1.0
    vis['traj']['link2']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['link2']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tl_l2))
    mat = meshcat.geometry.MeshLambertMaterial(color=0x07F100)
    mat.reflectivity = 1.0
    vis['traj']['linka']['points' + str(it)].set_object(
                meshcat.geometry.Sphere(0.02), mat)
    vis['traj']['linka']['points' + str(it)].set_transform(
                meshcat.transformations.translation_matrix(tl_la))
    

121.07271432876587
0.0
3.081364851518779


In [13]:
spp.Solve

<bound method PyCapsule.Solve of <pydrake.geometry.optimization.BsplineTrajectoryThroughUnionOfHPolyhedra object at 0x7f7ae1a7def0>>

In [14]:
#rrt cfg
limits = [np.array(q_low), np.array(q_high)]
mat = meshcat.geometry.MeshLambertMaterial(color= 0x000000 , wireframe=True)

def plotting_fn(parent, child, pos_samp, idx, mat, width):
    vis2['rrt']['tree']['line' + str(idx)].set_object( meshcat_line(parent.pos, child.pos, width), mat)
    
plotting_fn_handle = partial(plotting_fn, mat = mat, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrt_col_fn_handle = partial(collision, col_func_handle = col_func_handle)
    
RRT = rrt.RRT(start = start,
              goal = target,
              limits = limits,
              col_func_handle=rrt_col_fn_handle,
              max_extend_length=0.1,
              extend_steps=0.002,
              init_goal_sample_rate=0.1,
              goal_sample_rate_scaler=0.9,
              verbose = True, 
              plotcallback = plotting_fn_handle,
              sample_collision_free= False)

success, path = RRT.run(8000)
if success:
    mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
    mat.wireframeLinewidth = 2.0
    num_waypoints = len(path)
    for idx in range(num_waypoints-1):
        vis2['rrt']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
    traj= utils.PWLinTraj(path, 5.0)

0
it: 0 distance to target:  0.824 goalsample prob:  0.100
it: 1 distance to target:  0.746 goalsample prob:  0.129
it: 2 distance to target:  0.678 goalsample prob:  0.190
it: 3 distance to target:  0.606 goalsample prob:  0.254
it: 5 distance to target:  0.576 goalsample prob:  0.281
it: 28 distance to target:  0.574 goalsample prob:  0.283
1000
2000
3000
it: 3188 distance to target:  0.539 goalsample prob:  0.315
it: 3192 distance to target:  0.441 goalsample prob:  0.403
it: 3198 distance to target:  0.343 goalsample prob:  0.491
it: 3199 distance to target:  0.245 goalsample prob:  0.579
it: 3201 distance to target:  0.147 goalsample prob:  0.668
it: 3202 distance to target:  0.049 goalsample prob:  0.756
it: 3204 distance to target:  0.001 goalsample prob:  0.799
[RRT] Collision free path found in  3204  steps


In [15]:
#prm cfg
limits = [np.array(q_low), np.array(q_high)]
mat = meshcat.geometry.MeshLambertMaterial(color= 0x000000 , wireframe=True)

def plotting_fn(nodes, adjacency_list, mat, width):
    plt_idx = 0
    for node_idx in range(nodes.shape[0]):
        pos1 = nodes[node_idx, :]
        for edge_idx in range(len(adjacency_list[node_idx])): 
            pos2 = nodes[adjacency_list[node_idx][edge_idx], :]
            vis2['prm']['rm']['line' + str(plt_idx)].set_object( meshcat_line(pos1, pos2, width), mat)
            plt_idx +=1
            
plotting_fn_handle = partial(plotting_fn, mat = mat, width = 0.01)

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])

prm_col_fn_handle = partial(collision, col_func_handle = col_func_handle)

PRM = prm.PRM( 
            limits,
            num_points = 400,
            col_func_handle = prm_col_fn_handle,
            num_neighbours = 5, 
            dist_thresh = .5,
            num_col_checks = 10,
            verbose = True,
            plotcallback = plotting_fn_handle
            )

PRM.add_start_end(start, target)
PRM.plot()
path, sp_length = PRM.find_shortest_path()

mat = meshcat.geometry.MeshLambertMaterial(color= 0xFFF812 , wireframe=True)
mat.wireframeLinewidth = 2.0
num_waypoints = len(path)
for idx in range(num_waypoints-1):
    vis2['prm']['path']['path' + str(idx)].set_object( meshcat_line(path[idx], path[idx+1],width = 0.01), mat)
traj= utils.PWLinTraj(path, 5.0)

[PRM] Samples 0
[PRM] Samples 30
[PRM] Samples 60
[PRM] Samples 90
[PRM] Samples 120
[PRM] Samples 150
[PRM] Samples 180
[PRM] Samples 210
[PRM] Samples 240
[PRM] Samples 270
[PRM] Samples 300
[PRM] Samples 330
[PRM] Samples 360
[PRM] Samples 390
[PRM] Nodes connected: 0
[PRM] Nodes connected: 20
[PRM] Nodes connected: 40
[PRM] Nodes connected: 60
[PRM] Nodes connected: 80
[PRM] Nodes connected: 100
[PRM] Nodes connected: 120
[PRM] Nodes connected: 140
[PRM] Nodes connected: 160
[PRM] Nodes connected: 180
[PRM] Nodes connected: 200
[PRM] Nodes connected: 220
[PRM] Nodes connected: 240
[PRM] Nodes connected: 260
[PRM] Nodes connected: 280
[PRM] Nodes connected: 300
[PRM] Nodes connected: 320
[PRM] Nodes connected: 340
[PRM] Nodes connected: 360
[PRM] Nodes connected: 380
5 disconnected nodes [[155]
 [239]
 [338]
 [370]
 [400]]


In [14]:
#loop
substeps = 1000
utils.animate(traj, showres, substeps, 10*substeps)
            

In [20]:
def get_AABB_limits(hpoly):
    #im using snopt, sue me
    max_limits = []
    min_limits = []
    A = hpoly.A()
    b = hpoly.b()

    for idx in range(3):
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(3, 'x')
        cost = x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A@x,b))
        solver = SnoptSolver()
        result = solver.Solve(aabbprog)
        min_limits.append(result.get_optimal_cost()-0.01)
        aabbprog = MathematicalProgram()
        x = aabbprog.NewContinuousVariables(3, 'x')
        cost = -x[idx]
        aabbprog.AddCost(cost)
        aabbprog.AddConstraint(le(A@x,b))
        solver = SnoptSolver()
        result = solver.Solve(aabbprog)
        max_limits.append(-result.get_optimal_cost() + 0.01)
    return max_limits, min_limits

def plot_3d_poly(region, resolution, vis, name, mat = None):
    
    def inpolycheck(q0,q1,q2, A, b):
        q = np.array([q0, q1, q2])
        res = np.min(1.0*(A@q-b<=0))
        #print(res)
        return res
    
    aabb_max, aabb_min = get_AABB_limits(region)
    print('AABB:', aabb_min, aabb_max)
    col_hand = partial(inpolycheck, A=region.A(), b=region.b())
    vertices, triangles = mcubes.marching_cubes_func(tuple(aabb_min), 
                                                     tuple(aabb_max),
                                                     resolution, 
                                                     resolution, 
                                                     resolution, 
                                                     col_hand, 
                                                     0.5)
    if mat is None:
        mat = meshcat.geometry.MeshLambertMaterial(color=0x000000 , wireframe=True)
        mat.opacity = 0.3
    vis[name].set_object(
            meshcat.geometry.TriangularMeshGeometry(vertices, triangles),
            mat)

def plot_point(loc, radius, mat, vis, marker_id):
    vis['markers'][marker_id].set_object(
                meshcat.geometry.Sphere(radius), mat)
    vis['markers'][marker_id].set_transform(
                meshcat.transformations.translation_matrix(loc))

def Iris_fn(q_seed, iris_options, plant, plant_context, verbose):
    start_time = time.time()
    region = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    if verbose:
        print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
  
    return region, ellipse

IRISOPTS = IrisOptions()
IRISOPTS.require_sample_point_is_contained = True
IRISOPTS.iteration_limit = 60
IRISOPTS.enable_ibex = False

iris_hand = partial(Iris_fn, 
                    iris_options = IRISOPTS, 
                    plant = plant, 
                    plant_context = plant_context, 
                    verbose = True)

def get_closest_point_in_set(polys, sample, dim = 3):
    num_regions = len(polys) 
    prog = MathematicalProgram()
    x = prog.NewContinuousVariables(dim*num_regions, 'x')  
    cost = x - np.tile(sample, (1, num_regions)).squeeze()
    cost = cost@cost.T
    prog.AddCost(cost)
    for idx in range(num_regions):
        A = polys[idx].A()
        b = polys[idx].b()
        prog.AddConstraint(le(A@x[idx*dim:(idx + 1)*dim], b))
    solver = SnoptSolver()
    result = solver.Solve(prog)
    x_sol = result.GetSolution(x).reshape(num_regions,3)
    dists = np.linalg.norm(x_sol-sample, axis = 1)
    closest = np.argmin(dists)
    min_dist = dists[closest]
    min_point = x_sol[closest, :]
    
    return min_point, min_dist, x_sol, result

In [21]:
seed_points = []
polys = []
ells = []
region, ell = iris_hand(start)
polys.append(region)
ells.append(ell)
region, ell = iris_hand(target)
polys.append(region)
ells.append(ell)
idx = 0
marker_mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((255, 125, 125)) , wireframe=True)
for poly in polys:
    plot_3d_poly(poly, resolution= 20, vis = vis2['irisrrt'], name = 'poly_' + str(idx), mat = None )
    #plot_point(, radius = 0.05, mat = marker_mat , vis =vis2['irisrrt'], marker_id = str(idx))
    idx +=1

Time:   0.08 	Volume:   0.69 	Center: [-0.18154992 -1.03678813  1.1474287 ]
Time:   0.07 	Volume:   0.13 	Center: [ 0.64658822 -0.49381788  1.57702828]
AABB: [-0.6007769135705995, -1.71, -1.1951564470170672] [0.5450231335219549, 0.393947999411864, 2.01]
AABB: [0.08009357619380249, -1.1070183330589154, 1.0257797814829626] [1.0230006049199827, 0.0835758762380714, 2.01]


In [22]:
sample = np.array([0.2, -0.2, 0.3])
seed_points.append(sample)
marker_mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((255, 10, 50)) , wireframe=True)

plot_point(sample, radius = 0.05, mat = marker_mat , vis = vis2['irisrrt'], marker_id = 'sample')
new_seed_point, dists, x_sol, result =  get_closest_point_in_set(polys, sample)


marker_mat = meshcat.geometry.MeshLambertMaterial(color= rgb_to_hex((255, 0, 255)) , wireframe=True)
for idx in range(x_sol.shape[0]): 
    plot_point(x_sol[idx,:], radius = 0.05, mat = marker_mat , vis = vis2['irisrrt'], marker_id = 'sol' + str(idx))

region, ell = iris_hand(new_seed_point)
polys.append(region)
ells.append(ell)
plot_3d_poly(polys[-1], resolution= 20, vis = vis2['irisrrt'], name = 'poly_' + str(len(polys)), mat = None )
    

NameError: name 'rgb_to_hex' is not defined

In [24]:
result.is_success()

True

In [9]:
def Iris_fn(q_seed, iris_options, plant, plant_context, verbose):
    start_time = time.time()
    region = IrisInConfigurationSpace(plant, plant_context, q_seed, iris_options)
    ellipse = region.MaximumVolumeInscribedEllipsoid()
    if verbose:
        print("Time: %6.2f \tVolume: %6.2f \tCenter:" % (time.time() - start_time, ellipse.Volume()),
          ellipse.center(), flush=True)
  
    return region, ellipse

IRISOPTS = IrisOptions()
IRISOPTS.require_sample_point_is_contained = True
IRISOPTS.iteration_limit = 60
IRISOPTS.enable_ibex = False

iris_hand = partial(Iris_fn, 
                    iris_options = IRISOPTS, 
                    plant = plant, 
                    plant_context = plant_context, 
                    verbose = True)

def plot_callback(region, seed_point, pos_samp, region_id, vis):
    scale = np.clip(150 + region_id *105/5.0, a_min = 0, a_max = 255)
    mat = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((int(0.2*scale), 0, int(scale))) , wireframe=True)
    mat_seed = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((0, 0, 0)), wireframe=True)
    mat_samp = meshcat.geometry.MeshLambertMaterial(color= utils.rgb_to_hex((155, 0, 155)), wireframe=True)
    
    print('seed_point', seed_point)
    print('pos_samp', pos_samp)
    utils.plot_3d_poly(region = region,
                       resolution = 30,
                       vis = vis['regions'],
                       name = str(region_id),
                       mat = mat)
    utils.plot_point(seed_point,
                     radius = 0.05, 
                     mat = mat_seed, 
                     vis = vis['seed'], 
                     marker_id = str(region_id))
    
    utils.plot_point(pos_samp,
                     radius = 0.05, 
                     mat = mat_samp, 
                     vis = vis['samples'], 
                     marker_id = str(region_id))
    
plot_callback_handle = partial(plot_callback, vis = vis2['rrtiris']) 

def collision(pos, col_func_handle):
    return col_func_handle(pos[0], pos[1], pos[2])
    
rrti_col_fn_handle = partial(collision, col_func_handle = col_func_handle)

In [13]:
RRTI = rrtiris.RRTIRIS(start = start, 
                       goal = target , 
                       limits = [np.array(q_low), np.array(q_high)], 
                       iris_handle = iris_hand,
                       col_func_handle = rrti_col_fn_handle, 
                       init_goal_sample_rate = 0.05,
                       goal_sample_rate_scaler = 0.1,
                       verbose = False,
                       plotcallback = plot_callback_handle,
                       sample_collision_free = True
                       )

Time:   0.05 	Volume:   0.69 	Center: [-0.18154992 -1.03678812  1.14742869]
seed_point [ 0.2 -1.6  1.5]
pos_samp [ 0.2 -1.6  1.5]


In [14]:
RRTI.run(12)

0
[RRTIRIS] pos_samp [-1.02545864 -0.53364628 -1.30121701]
Time:   0.12 	Volume:   1.48 	Center: [-0.02302292 -0.81321198  0.25977681]
1.7384768607043153
seed_point [-0.03878901 -0.63782678  0.        ]
pos_samp [-0.11511209 -0.0848522   0.42576262]
1
[RRTIRIS] pos_samp [ 0.56104285 -0.24839976 -1.75700988]
Time:   0.16 	Volume:   1.52 	Center: [ 0.19923839 -0.51555753 -0.16204038]
1.7384768607043153
seed_point [0. 0. 0.]
pos_samp [ 0.56104285 -0.24839976 -1.75700988]
2
[RRTIRIS] pos_samp [ 1.09337865 -1.35813945  0.81679103]
Time:   0.12 	Volume:   1.48 	Center: [-0.02302332 -0.81321198  0.2597791 ]
[RRT IRIS] Region check failed
3
[RRTIRIS] pos_samp [-0.3835056  -0.19183109  0.9675242 ]
Time:   0.16 	Volume:   1.52 	Center: [ 0.19923926 -0.51555695 -0.16204142]
[RRT IRIS] Region check failed
4
[RRTIRIS] pos_samp [0.46969748 0.3618522  1.78396628]
Time:   0.16 	Volume:   1.52 	Center: [ 0.19923867 -0.51555738 -0.16204075]
[RRT IRIS] Region check failed
5
[RRTIRIS] pos_samp [-0.3408130

In [115]:
cost.

<Expression "(pow((-0.69999999999999996 + x(2)), 2) + pow((-0.69999999999999996 + x(5)), 2) + pow((-0.20000000000000001 + x(0)), 2) + pow((-0.20000000000000001 + x(3)), 2) + pow((0.20000000000000001 + x(1)), 2) + pow((0.20000000000000001 + x(4)), 2))">